## This code for training the random forest model was written using the sources below
### https://github.com/MohamedEl1/Dublin-bikes-web-app/blob/master/Data%20Analytics/machine_learning%20.ipynb
### https://www.geeksforgeeks.org/ml-label-encoding-of-datasets-in-python/

In [14]:

import pickle
import pandas as pd
import numpy as np
import sqlalchemy
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import json
import requests
import time
from IPython.display import display
import sys

In [2]:

url = "dbbikes.cvvkn6jkfbdf.eu-west-1.rds.amazonaws.com"
port = "3306"
db = "dbbikes"
user = "SEGroup25"
password = "SEGroup25?"



In [3]:

engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{db}", echo=True)


In [4]:

availability_sql = pd.read_sql("select * from dbbikes.availability", engine)

2023-04-20 21:44:19,878 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-20 21:44:19,881 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 21:44:19,914 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-20 21:44:19,915 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 21:44:19,934 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-20 21:44:19,936 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 21:44:19,998 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-20 21:44:20,002 INFO sqlalchemy.engine.Engine [generated in 0.00304s] ('dbbikes', 'select * from dbbikes.availability')
2023-04-20 21:44:20,039 INFO sqlalchemy.engine.Engine select * from dbbikes.availability
2023-04-20 21:44:20,042 INFO sqlalchemy.engine.Engine [raw sql] ()


In [5]:

weather_sql = pd.read_sql("select * from dbbikes.weather", engine)

2023-04-20 21:44:23,394 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-20 21:44:23,396 INFO sqlalchemy.engine.Engine [cached since 3.398s ago] ('dbbikes', 'select * from dbbikes.weather')
2023-04-20 21:44:23,435 INFO sqlalchemy.engine.Engine select * from dbbikes.weather
2023-04-20 21:44:23,436 INFO sqlalchemy.engine.Engine [raw sql] ()


In [6]:

weather_sql.shape[0]

2753

In [7]:

availability_sql.shape

(691584, 4)

In [8]:

availability_sql.head(5)

,available_bike_stands,available_bikes,last_update,number
0,23,7,2147483647,42
1,14,6,2147483647,30
2,21,12,2147483647,54
3,21,14,2147483647,108
4,14,16,2147483647,20


In [9]:

df_trimmed = availability_sql.groupby('number').tail(weather_sql.shape[0])


df_trimmed = df_trimmed.reset_index(drop=True)

In [10]:

df_trimmed.shape

(313995, 4)

In [11]:

df_trimmed['index'] = df_trimmed.groupby('number').cumcount()
merged_df = df_trimmed.join(weather_sql,on='index', how='left')
merged_df.head()

,available_bike_stands,available_bikes,last_update,number,index,time,cord_long,cord_lat,weather_main,main_temp,main_humidity,wind_speed,wind_degree,sys_sunset,sys_sunrise
0,0,1,2147483647,507,0,2023-03-30 13:53:20,-6.2672,53.344,Clouds,14.61,71.0,4.12,220.0,2023-03-30 18:54:48,2023-03-30 06:04:13
1,0,1,2147483647,507,1,2023-03-30 13:55:03,-6.2672,53.344,Clouds,14.61,71.0,4.12,220.0,2023-03-30 18:54:48,2023-03-30 06:04:13
2,0,1,2147483647,507,2,2023-03-30 14:11:04,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13
3,0,1,2147483647,507,3,2023-03-30 14:11:26,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13
4,0,1,2147483647,507,4,2023-03-30 14:15:02,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13


In [12]:

label_encoder = preprocessing.LabelEncoder()
merged_df['weather_main'] = label_encoder.fit_transform(merged_df['weather_main'])

In [15]:

merged_df.tail()

,available_bike_stands,available_bikes,last_update,number,index,time,cord_long,cord_lat,weather_main,main_temp,main_humidity,wind_speed,wind_degree,sys_sunset,sys_sunrise
313990,20,0,2147483647,39,2752,2023-04-20 11:45:03,-6.2672,53.344,1,11.26,69.0,8.23,100.0,2023-04-20 19:32:46,2023-04-20 05:14:57
313991,20,20,2147483647,83,2752,2023-04-20 11:45:03,-6.2672,53.344,1,11.26,69.0,8.23,100.0,2023-04-20 19:32:46,2023-04-20 05:14:57
313992,35,5,2147483647,92,2752,2023-04-20 11:45:03,-6.2672,53.344,1,11.26,69.0,8.23,100.0,2023-04-20 19:32:46,2023-04-20 05:14:57
313993,12,18,2147483647,21,2752,2023-04-20 11:45:03,-6.2672,53.344,1,11.26,69.0,8.23,100.0,2023-04-20 19:32:46,2023-04-20 05:14:57
313994,17,13,2147483647,88,2752,2023-04-20 11:45:03,-6.2672,53.344,1,11.26,69.0,8.23,100.0,2023-04-20 19:32:46,2023-04-20 05:14:57


In [16]:

merged_df['month'] = merged_df['time'].dt.month
merged_df['dayofweek'] = merged_df['time'].dt.dayofweek
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['minute'] = merged_df['time'].dt.minute

In [17]:
print(merged_df.isnull().sum().sum())

0


In [18]:

merged_df.dtypes

available_bike_stands             int64
available_bikes                   int64
last_update                       int64
number                            int64
index                             int64
time                     datetime64[ns]
cord_long                       float64
cord_lat                        float64
weather_main                      int64
main_temp                       float64
main_humidity                   float64
wind_speed                      float64
wind_degree                     float64
sys_sunset               datetime64[ns]
sys_sunrise              datetime64[ns]
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

In [19]:
merged_df.shape

(313995, 19)

In [20]:

X = merged_df[['number','month','hour','minute','weather_main','main_temp','main_humidity','wind_speed','dayofweek']]
Y = merged_df.available_bikes
X_train, X_test,y_train,y_test = train_test_split(X, Y, test_size=0.3,random_state=0)
bikes_model = RandomForestRegressor(n_estimators=15)

bikes_model.fit(X_train, y_train)
y_pred = bikes_model.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})


print('Predictions for Bikes: ')
print(df)
print()


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("Accuracy score : ",metrics.r2_score(y_test, y_pred))


with open('bikes_model.pkl', 'wb') as handle:
    pickle.dump(bikes_model, handle, pickle.HIGHEST_PROTOCOL)

Predictions for Bikes: 
        Actual  Predicted
115168      25  26.333333
186416      11  11.000000
229124      11  11.066667
232423       2   1.266667
249669       1   1.000000
...        ...        ...
244943       6   4.533333
32521        6   7.066667
66391        6   6.666667
137557      11  12.666667
48277       13  14.133333

[94199 rows x 2 columns]

Mean Absolute Error: 0.965132670557675
Mean Squared Error: 4.465499337071113
Root Mean Squared Error: 2.1131728128743075
Accuracy score :  0.9494020256425311


In [21]:
A = merged_df[['number','month','hour','minute','weather_main','main_temp','main_humidity','wind_speed','dayofweek']]
B = merged_df.available_bike_stands
A_train, A_test,b_train,b_test = train_test_split(A, B, test_size=0.3,random_state=0)
stands_model = RandomForestRegressor(n_estimators=15)

stands_model.fit(X_train, b_train)
b_pred =stands_model.predict(A_test)
df = pd.DataFrame({'Actual': b_test, 'Predicted': b_pred})

print('Predictions for Bike Stands: ')
print(df)
print()

print('Mean Absolute Error:', metrics.mean_absolute_error(b_test, b_pred))
print('Mean Squared Error:', metrics.mean_squared_error(b_test, b_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(b_test, b_pred)))
print ("Accuracy Score : ",metrics.r2_score(b_test, b_pred))

with open('stands_model.pkl', 'wb') as handle:
    pickle.dump(stands_model, handle, pickle.HIGHEST_PROTOCOL)

Predictions for Bike Stands: 
        Actual  Predicted
115168      15  13.733333
186416      19  19.000000
229124       9   8.733333
232423      28  29.000000
249669      39  39.000000
...        ...        ...
244943      14  14.000000
32521       34  34.466667
66391       33  32.066667
137557      18  16.533333
48277        6   8.466667

[94199 rows x 2 columns]

Mean Absolute Error: 0.9562847767231042
Mean Squared Error: 4.541312889135377
Root Mean Squared Error: 2.1310356376971686
Accuracy Score :  0.9609815047655029
